## SCD Type 2 and Time Travel Demo

##### Before Ingesting New Data

In [2]:
%%sql

-- Check the primary position of Marian Hossa
SELECT * FROM dim_players
WHERE first_name = 'Marian' AND last_name = 'Hossa';

-- Check if player_name Fourcast Analytics exists in current table
SELECT * FROM dim_players
WHERE first_name = 'Fourcast';

StatementMeta(, e71d3942-1e44-43c1-b81f-1e53bd273b37, 5, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 15 fields>

<Spark SQL result set with 0 rows and 15 fields>

In [3]:
# Check latest Delta log version

spark.sql("DESCRIBE HISTORY dim_players").select(
    "version", "timestamp", "operation", "operationParameters", "userName"
).orderBy("version", ascending=False).show(truncate=False)


StatementMeta(, e71d3942-1e44-43c1-b81f-1e53bd273b37, 7, Finished, Available, Finished)

+-------+-----------------------+---------+--------------------------------------+--------+
|version|timestamp              |operation|operationParameters                   |userName|
+-------+-----------------------+---------+--------------------------------------+--------+
|0      |2025-10-07 13:03:27.869|WRITE    |{mode -> Overwrite, partitionBy -> []}|NULL    |
+-------+-----------------------+---------+--------------------------------------+--------+



##### After Ingesting New Data

In [1]:
%%sql

-- Check the primary position of Marian Hossa
SELECT * FROM dim_players
WHERE first_name = 'Marian' AND last_name = 'Hossa';

-- Check if player_name Fourcast Analytics exists in current table
SELECT * FROM dim_players
WHERE first_name = 'Fourcast';

StatementMeta(, e1e01055-a665-4a74-a8fd-b11f5f64f245, 3, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 15 fields>

<Spark SQL result set with 1 rows and 15 fields>

In [2]:
# Check latest Delta log version

spark.sql("DESCRIBE HISTORY dim_players").select(
    "version", "timestamp", "operation", "operationParameters", "userName"
).orderBy("version", ascending=False).show(truncate=False)

StatementMeta(, e1e01055-a665-4a74-a8fd-b11f5f64f245, 5, Finished, Available, Finished)

+-------+-----------------------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|version|timestamp              |operation|operationParameters                                                                                                                                                                                |userName|
+-------+-----------------------+---------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|2      |2025-10-07 23:45:20.557|MERGE    |{predicate -> ["((player_id#2612 = player_id#2539) AND (is_current#2623 = true))"], matchedPredicates -> [], notMatchedPredicates -> [{"actionType":"insert"}], notMatchedBySourcePredicates -> []}|NULL    |
|1  

##### Time Travel Demo (Reading)

In [3]:
%%sql
------ Time Travel to Previous Version -------

-- Check for Marian Hossa
SELECT * FROM delta.`Tables/dim_players` VERSION AS OF 1
WHERE first_name = 'Marian' AND last_name = 'Hossa';

-- Check for Fourcast Analytics
SELECT * FROM delta.`Tables/dim_players` VERSION AS OF 1
WHERE first_name = 'Fourcast';

StatementMeta(, e1e01055-a665-4a74-a8fd-b11f5f64f245, 7, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 15 fields>

<Spark SQL result set with 0 rows and 15 fields>

##### Time Travel Demo (Rollback)

In [ ]:
%%sql
-- Recreate the table from version 1
CREATE OR REPLACE TABLE dim_players AS
SELECT *
FROM delta.`Tables/dim_players` VERSION AS OF 1;